### Auto Acquisition at Feature Positions

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os


from odemis import model
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from odemis import model
from odemis.acq.move import MicroscopePostureManager, POSITION_NAMES
from odemis.acq.stream import FluoStream
from odemis.acq.feature import load_project_data, acquire_at_features, FEATURE_DEACTIVE
from odemis.util.comp import generate_zlevels
import random

microscope = model.getMicroscope()
pm = MicroscopePostureManager(microscope)

stage = model.getComponent(role="stage")
stage_bare = model.getComponent(role="stage-bare")

ccd = model.getComponent(role="ccd")
light = model.getComponent(role="light")
focus = model.getComponent(role="focus")
em_filter = model.getComponent(role="filter")
fm_stream = FluoStream(
    name="FM",
    detector=ccd,
    dataflow=ccd.data,
    emitter=light,
    em_filter=em_filter,
    focuser=focus,
    # opm=self._main_data_model.opm,
    # detvas={"exposureTime"},
)


In [ ]:
# PROJECT DATA
PROJECT_PATH = "/home/patrick/Pictures/20240723-61"

project_data = load_project_data(PROJECT_PATH)

features = project_data["features"]

for f in features:
    print(f.name.value, f.status.value, f.pos.value)


In [ ]:

initial_position = None
if initial_position is None:
    initial_position = stage.position.value

f = stage.moveAbs(initial_position)
f.result()
print(f"Current Stage Position: {initial_position}")

# move to FM IMAGING
current_posture = pm.getCurrentPostureLabel()
print(f"Current Posture: {POSITION_NAMES[current_posture]}")

ZSTACK = random.choice([True, False])

zlevels = {}  # Stream: List[float]
if ZSTACK:
    zmin = -2e-6
    zmax = 2e-6
    zstep = 1e-6
    levels = generate_zlevels(focuser=focus, zrange=(zmin, zmax), zstep=zstep)

    zlevels = {s: levels for s in [fm_stream] if isinstance(s, (FluoStream))}

print(f"Z-stack enabled: {ZSTACK}")
if ZSTACK:
    # get the number of z-levels
    n_zlevels = len(list(zlevels.values())[0])
    print(f"Number of z-levels: {n_zlevels}")
    print(f"Z-levels", zlevels.values())

# TODO: export data
# TODO: time estimattion
# TODO: display data in UI

filename = os.path.join(PROJECT_PATH, "test-auto-acq.ome.tiff")

features[1].status.value = FEATURE_DEACTIVE

f = acquire_at_features(features, stage, focus, [fm_stream], zlevels=zlevels, filename=filename)
f.result()

In [ ]:
focus.moveAbs({"z": 1.69e-3})

focus.position.value